# Table of Contents
* Problem Analysis
* Exploratory Data Analysis
* Data Preprocessing
* Baseline Model Performance
* Hyperparameter Tuning
* Model Results & Interpretation

## Questions & Answers
- **Tell us how you validate your model, which, and why you chose such evaluation technique(s)**
    * Validation is done on a held-out test set (30% of dataset), to prevent data leakage.
    * Given that the evaluation metric chosen for the competition is AUCROC, I used the same evaluation metric for validation on kfold cross validation. For reasons explained below, this is a good metric to use.
- **What is AUC? Why do you think AUC was used as the evaluation metric for such a problem? What are other metrics that you think would also be suitable for this competition?**
    * AUC is the area under the receiver operating characteristic curve, which **measures the ability of a classifier to distinguish between classes across different threshold values**.
    * Poor metrics to use include accuracy, which is unsuitable due to the imbalanced nature of the target variable (baseline model predicting 0s will get 94% accuracy).
    * For this problem, false negatives are more severe than false positives, as it is more detrimental to the bank if a customer defaults, than for it to lose a customer that would not have defaulted. Also, a human can be in the loop to verify positive cases, hence the model can act as an initial filter for possible defaulters. As such, **recall** is more important than precision.
    * However, using only recall will cause the model to be imprecise. To balance precision with recall, we can use the **F-measure**, include the F1 score, which weights precision and recall equally. Since recall is more important than precision, we can use **F-Beta** for Beta > 1, e.g. Beta = 2.
    * Varying the classifier's threshold value allows us to tradeoff precision with recall and vice versa, and **AUCROC** is a metric that captures this notion, ranging from 0.5 (baseline model) to 1 (higher is better). Hence, we can plot the ROC curve and select for a threshold where the model's recall is sufficiently high for our business case, while remaining sufficiently precise.
- **What insight(s) do you have from your model? What is your preliminary analysis of the given dataset?**
    * Detailed analysis shown in notebook below
- **Can you get into the top 100 of the private leaderboard, or even higher?**
    * Validation AUCROC: 86.6%, Test set (30% of dataset) AUCROC: 86.6%, Kaggle AUCROC: 86.083%
    * Kaggle performance: **Public: 141 / 924 (top 15%)**, Private: 519 / 924 (top 56%)

# Problem Analysis

## Problem Statment
Banks play a crucial role in market economies. They decide who can get finance and on what terms and can make or break investment decisions. For markets and society to function, individuals and companies need access to credit. 

Credit scoring algorithms, which make a guess at the probability of default, are the method banks use to determine whether or not a loan should be granted. This competition requires participants to improve on the state of the art in credit scoring, by **predicting the probability that somebody will experience financial distress in the next two years**.

The goal of this competition is to **build a model that borrowers can use to help make the best financial decisions**. Historical data are provided on 250,000 borrowers.

## Key Observations

1. Goal is for borrowers to understand what they can do to improve their credit score. Hence, **model explainability is important**, where interpretable model outputs give actionable insights for borrowers.
2. Problem is a **binary classification** problem of whether customer will default in the next 2 years.
3. From the bank's perspective: **False negatives are more costly than false positives**, as they lose more money and create negative social impact when customers default on their loans (a la Global Financial Crisis, 2008), hence outweighing the loss of interest earned from customer loans. Hence **Recall > Precision in importance**. To mitigate, this model can have a human-in-the-loop to process credit application rejections to combat unfairness complaints.
4. As model as broader societal implications (rich get richer), **model fairness is of utmost importance**. Hence, using features that can be deemed unfair, e.g. gender, religion, nationality, age, should be minimised.

In [ ]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, plot_roc_curve

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline

from xgboost import XGBClassifier

import shap

import warnings
warnings.filterwarnings('ignore')

sns.set_theme(style="darkgrid")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
root_folder = '/kaggle/input/GiveMeSomeCredit/'

# Exploratory Data Analysis

## Loading & Visualizing data

10 features, 150k observations: **Moderate sized dataset with few features** (more long than wide)

In [ ]:
# removing unused first (index) column
df = pd.read_csv(root_folder+'cs-training.csv').iloc[:,1:]
df

In [ ]:
target_variable = "SeriousDlqin2yrs"

## Data Dictionary
Features can be classified into:
   1. Historical late repayments in the last 2 years(3 window periods of 30-59, 60-89, >=90)
   2. Financial Obligations (NumberOfOpenCreditLinesAndLoans, NumberOfDependents, DebtRatio, RevolvingUtilizationOfUnsecuredLines)
   3. Financial Capabilities (MonthlyIncome)
   4. Demographics (Age)

| Variable Name                        | Description                                                                                                                                              | Type       |
| ------------------------------------ | -------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------- |
| SeriousDlqin2yrs                     | Person experienced 90 days past due delinquency or worse                                                                                                 | Y/N        |
| RevolvingUtilizationOfUnsecuredLines | Total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits | percentage |
| age                                  | Age of borrower in years                                                                                                                                 | integer    |
| NumberOfTime30-59DaysPastDueNotWorse | Number of times borrower has been 30-59 days past due but no worse in the last 2 years.                                                                  | integer    |
| DebtRatio                            | Monthly debt payments, alimony,living costs divided by monthy gross income                                                                               | percentage |
| MonthlyIncome                        | Monthly income                                                                                                                                           | real       |
| NumberOfOpenCreditLinesAndLoans      | Number of Open loans (installment like car loan or mortgage) and Lines of credit (e.g. credit cards)                                                     | integer    |
| NumberOfTimes90DaysLate              | Number of times borrower has been 90 days or more past due.                                                                                              | integer    |
| NumberRealEstateLoansOrLines         | Number of mortgage and real estate loans including home equity lines of credit                                                                           | integer    |
| NumberOfTime60-89DaysPastDueNotWorse | Number of times borrower has been 60-89 days past due but no worse in the last 2 years.                                                                  | integer    |
| NumberOfDependents                   | Number of dependents in family excluding themselves (spouse, children etc.)                                                                              | integer    |



## Visualizing distributions of features

Initial exploration on the dependent variables, to get an intuition on what models will perform well on this problem, and on how to prepare the dataset for training.

In [ ]:
df.describe(percentiles=[.25,.5,.75,.9,.95,.99,.999])

In [ ]:
g = sns.PairGrid(df.sample(150), diag_sharey=False)
g.map_upper(sns.scatterplot, s=15)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=2);

In [ ]:
sns.kdeplot(data=df.sample(150), x="RevolvingUtilizationOfUnsecuredLines", y=target_variable, 
            thresh=.1)

In [ ]:
sns.boxplot(x="NumberOfDependents", y="age",
            hue="SeriousDlqin2yrs", palette=["m", "g"],
            data=df)

### Checking target variable distribution

Looking at the distribution of the target variable, this problem is a binary classification problem of classes 0 or 1 denoting if cusomter will default in 2 years, with class 1 being the minority at 6.684%

Hence, this is an **imbalanced learning problem**

In [ ]:
ax = sns.countplot(x=target_variable, data=df)
plt.title('Distribution of target variable')
plt.xlabel(target_variable)
plt.ylabel('Frequency')

for p in ax.patches:
        ax.annotate('{:,}'.format(p.get_height()), (p.get_x()+0.1, p.get_height()+50))

### Checking for NAs
Importance: Many learning algorithms cannot handle missing values, hence we need to handle missing values by dropping these observations, or imputing them. Imputing will increase the bias in our model, but dropping too many observations leads to a much smaller dataset for training. We will explore the dataset to drive our imputation strategy.

Only **2 features have nulls** (Monthly income and number of dependents). This could be due to customers not wanting to declare these personal information.

In [ ]:
df.info()

In [ ]:
features_with_na = df.isna().sum()[df.isna().sum()>0]
features_with_na.sort_values(ascending=False)

In [ ]:
sns.boxplot(x=target_variable, y="MonthlyIncome", data=df.sample(150),
            whis=[0, 100], width=.6, palette="vlag");

In [ ]:
sns.boxplot(x=target_variable, y="NumberOfDependents", data=df.sample(150),
            whis=[0, 100], width=.6, palette="vlag");

We have 29k observations with >= 1 feature having NA values, corresponding to **20% of the dataset, which is quite significant**. We will **have to impute these NA values** rather than dropping observations with NA. **Monthly Income and NumberOfDependents** have right tail skews, so **median imputation** is preferred to be robust to outliers.

In [ ]:
(df.isna().sum(axis=1)[df.isna().sum(axis=1)>0]
         .reset_index().rename(columns={0:'number_of_na'})
         .groupby('number_of_na')
         .count().rename(columns={'index':'number_of_observations'}))

### Detecting outliers

Many of the financial features have outliers (extreme outliers = > 3* interquartile range), with a right-tail skew. Hence, we should **either use models that are robust to outliers (e.g. tree-based models), or transform the features accordingly (e.g. Box-Cox transformation)**.

In [ ]:
def get_outlier_counts(df, outlier_threshold=1.5):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1

    outlier_counts = ((df < (Q1 - outlier_threshold * IQR)) | (df > (Q3 + outlier_threshold * IQR))).sum()
    return outlier_counts[outlier_counts>0].sort_values(ascending=False)

get_outlier_counts(df, outlier_threshold=3)

In [ ]:
df.describe(percentiles=[.25,.5,.75,.9,.95,.99,.999])

### Checking for correlations

1. Detecting multicollinearity: **30, 60 and 90 days past due are highly correlated with each other** (close to 1). This may impact performance if the learning algorithm used assumes independence in dependent variables. We can **either use models that are robust to multicollinearity (e.g. tree-based models), or use feature selection / regularization methods to use just one of these features.**

2. **30/60/90 days late payment** features have slight positive correlation with the target variable (0.12), while **age** has slight negative correlation with the target variable (-0.12). These are likely **useful features for predicting the target**.

In [ ]:
fig = plt.figure(figsize = [12,8])
sns.heatmap(df.corr(), cmap=sns.diverging_palette(240, 10, n=9),annot=True, center=0)
plt.title("Correlation Matrix", fontsize=12);

### Handling duplicates

Getting counts of number of rows by the number of duplicates: Most severe = 1 observations with 12 instances. Overall, the training set has not too many duplicates, hence will not likely bias the model much. These duplicates are only a problem if they came from non-random error, e.g. duplicate entry into database. In a real scenario, we should dig deeper to identify if these are errors or coincidences. For this problem, we **will not remove these duplicates and assume they are correct**.

In [ ]:
len(df[df.duplicated()])

In [ ]:
(df.groupby(df.columns.tolist(),as_index=False)
     .size()['size'].value_counts())

# Feature Engineering

## Train test split

**Train test split** is important to leave out a hold-out set (**30%** of dataset size) for evaluation, to **prevent data leakage**.

In [ ]:
X, y = df.iloc[:,1:], df.loc[:,target_variable]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Data Preprocessing

Based on EDA, we have found the need for imputation and scaling. Hence, we assemble a **pipeline of imputation (median) and scaling (Box-Cox via PowerTransformer)**, testing out different classifiers to compare model performance. Based on experimentation, model performance when training with SMOTE for synthetic oversampling of the minority class did not significantly improve performance.


In [ ]:
def get_pipeline(classifier, random_seed=42):
    """Takes in a classifier, returns the entire data preprocessing pipeline (imblearn pipeline)"""
    return Pipeline([
                    ('imputer', SimpleImputer(strategy='median')),
                    # SMOTE(sampling_strategy='minority', random_state=random_seed), -> removed due to poor performance
                    ('scaler', PowerTransformer()),
                    ('clf', classifier)
                    ])

## Baseline models

We train several baseline models using different learning algorithms to get a sense of what learning algorithm best performs during **k-fold cross validation**. In the interest of time, we perform model selection with minimal hyperparameter tuning, and **select the model with best performance on cross-validation score**. Ideally,this selection process happens after hyperparameter tuning for all of the models.

In [ ]:
def train_models(classifiers, num_folds=3, random_seed=42):
    # models = dict()
    results = dict()
    for classifier in classifiers:
        curr_time = time.time()
        model = get_pipeline(classifier, random_seed=random_seed)
        kfold = KFold(n_splits=num_folds)
        print(f'Training {classifier.__class__.__name__}')
        score = cross_val_score(model, X_train, y_train, cv=kfold, scoring='roc_auc')
        print("%0.2f AUC with a standard deviation of %0.2f" % (score.mean(), score.std()))
        # models[classifier.__class__.__name__] = model
        results[classifier.__class__.__name__] = score
        print(f'Training took {round(time.time()-curr_time)} seconds')
    print('Training complete')
    return results


classifiers = [
    GaussianNB(),
    KNeighborsClassifier(5),
    LogisticRegression(),
    RandomForestClassifier(),
    XGBClassifier(),
]
results = train_models(classifiers)
results

## Hyperparameter tuning

Based on the baseline model performances, **XGBoost algorithm** performed the best (**highest AUC with lowest standard deviation across 3 folds**). To improve performance, we tune the model hyperparameters using **randomized search**, which does random search over the range of hyperparameters. This is faster than grid search, which exhaustively searches over all possible hyperparameters in the range provided.

In [ ]:
model = get_pipeline(XGBClassifier(n_jobs=-1))

xgb_hyperparams = {
    'clf__max_depth' : np.arange(4,10,1),
    'clf__learning_rate': [0.01, 0.1, 0.2, 0.3],
    'clf__n_estimators' : np.arange(400,1000,100),
    'clf__subsample': np.arange(0.5,1,0.1),
    'clf__scale_pos_weight': [10,15,20] # Imbalanced ratio of 94:6, hence add weight to negative class
    
}

clf = RandomizedSearchCV(estimator=model,
                         param_distributions=xgb_hyperparams,
                         scoring='roc_auc',
                         n_iter=20,
                         verbose=2)

clf.fit(X_train, y_train)

print("Best parameters:", clf.best_params_)
print("Best score: ", clf.best_score_)

## Final model

We evaluate model performance using ROC AUC, which is the **area under the receiver operating characteristic curve**. 

In [ ]:
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    predicted_proba = model.predict_proba(X_test)

    # Default score: Mean accuracy, which is not a good evaluation metric for a imbalanced dataset problem
    print('accuracy', model.score(X_test, y_test))
    print('confusion_matrix')
    print(confusion_matrix(y_test, predictions))
    print('roc_auc',roc_auc_score(y_test, predicted_proba[:,1]))
    print('classification_report')
    print(classification_report(y_test, predictions))
    plot_roc_curve(model, X_test, y_test)

From the model classification report, we see that its **recall for the positive class is high at 84%**. This is preferred, as per business use case where **type II errors are more costly than type I errors**. Overall, its **ROC is 86.6% on the held out test set**, which is similar to the average **86.4% on cross-validation set**, showing that the **model can perform well with out-of-sample data (i.e. is not overfitted)**.

In [ ]:
model = get_pipeline(XGBClassifier(max_depth=4,
                                   learning_rate=0.01,
                                   n_estimators=800,
                                   subsample=0.5,
                                   scale_pos_weight=20,
                                   n_jobs = -1))

model.fit(X_train, y_train)
evaluate_model(model, X_test, y_test)

### Model explainability

From the feature importances, we see that **current utilization of credit (RevolvingUtilizationOfUnsecuredLines), 30-59 and >90 days late are the most useful features**. This **corresponds well with our intuition** that customers with history of frequent late payments and with are more likely to be delinquent on loan repayment in the future.

**For customers**, the actionable insights are that **the feature importances can be interpreted as a priority list of issues to fix for improving their credit score**. For example, ensuring one has low utilization of unsecured lines is more impactful in improving credit scoring than increasing their debt ratio or monthly income.

In [ ]:
(pd.DataFrame(model['clf'].feature_importances_, X_train.columns)
     .reset_index()
     .rename({'index':'features', 0:'feature_importance'}, axis=1)
     .sort_values(by=['feature_importance'], ascending=False))

The **SHAP model quantifies the contributions that each feature brings to generate the model outputs**. Plotting the average contributions across all predictions, we can visualize how much each value affect's one's probability of the model predicting one's delinquency in the next 2 years (and hence credit score). For example, we see that **higher RevolvingUtilizationOfUnsecuredLines leads to higher probability of delinquency**, as visualized by red scatter points having positive SHAP values.

In [ ]:
def plot_shap(model, df):
    """Plots shapely values for model explainability"""
    X_sampled = df.sample(100)

    explainer = shap.TreeExplainer(model['clf'])
    observations = model['scaler'].transform(model['imputer'].transform(X_sampled))
    
    shap_values = explainer.shap_values(observations)
    shap.summary_plot(shap_values, X_sampled)
    
plot_shap(model, X_test)

# Saving outputs

Submitting output to [kaggle](https://www.kaggle.com/c/GiveMeSomeCredit/leaderboard), the **XGBoost AUCROC for the test results were 0.86083%**. On the public leaderboard, this would be **position 141 / 924 (top 15%)**.

In [ ]:
def get_output(model, test_df):
    output_predictions = model.predict_proba(test_df.iloc[:,2:])
    output_df = pd.DataFrame(output_predictions[:,1]).rename(columns={0:'probability'})
    output_df['id'] = range(1,len(output_df)+1)
    output_df = output_df.loc[:,['id','probability']]
    return output_df

test_df = pd.read_csv(root_folder+'cs-test.csv')
output_df = get_output(model, test_df)

output_df.to_csv('output_predictions.csv', index=False)
output_df